In [4]:
import os
import random
import re
import string

import nltk
import numpy as np
import pandas as pd

import gensim
from gensim.models import Word2Vec, KeyedVectors

from nltk import word_tokenize
from nltk.corpus import stopwords
import spacy

nlp = spacy.load('en_core_web_lg')
#nltk.download("stopwords")

In [5]:
#!pip install gensim
#!pip install --upgrade pip

In [6]:
df_raw=pd.read_excel("Injury Triage 2022.xlsx")

In [7]:
def clean_text(text, tokenizer, stopwords):
    """Pre-process text and generate tokens

    Args:
        text: Text to tokenize.

    Returns:
        Tokenized text.
    """
    text = str(text).lower()  # Lowercase words
    text = re.sub(r"\[(.*?)\]", "", text)  # Remove [+XYZ chars] in content
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    text = re.sub(r"\w+…|…", "", text)  # Remove ellipsis (and last word)
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  # Replace dash between words
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)  # Remove punctuation

    tokens = tokenizer(text)  # Get tokens from text
    tokens = [t for t in tokens if not t in stopwords]  # Remove stopwords
    tokens = ["" if t.isdigit() else t for t in tokens]  # Remove digits
    tokens = [t for t in tokens if len(t) > 1]  # Remove short tokens
    return tokens

In [8]:
custom_stopwords = set(stopwords.words("english"))
#text_columns = ["title", "description", "content"]

df = df_raw.copy()
# df["content"] = df["content"].fillna("")

# for col in text_columns:
#     df[col] = df[col].astype(str)

# Create text column based on title, description, and content
#df["text"] = df[text_columns].apply(lambda x: " | ".join(x), axis=1)
df["tokens"] = df["Description of Event"].map(lambda x: clean_text(x, word_tokenize, custom_stopwords))

# Remove duplicated after preprocessing
_, idx = np.unique(df["tokens"], return_index=True)
df = df.iloc[idx, :]

# Remove empty values and keep relevant columns
df = df.loc[df.tokens.map(lambda x: len(x) > 0), ["Description of Event", "tokens"]]

docs = df["Description of Event"].values
tokenized_docs = df["tokens"].values

print(f"Original dataframe: {df_raw.shape}")
print(f"Pre-processed dataframe: {df.shape}")

Original dataframe: (720, 19)
Pre-processed dataframe: (720, 2)


In [9]:
pd.set_option('display.max_colwidth',None)

In [10]:
df.head()

,Description of Event,tokens
301,"AT 12:10pm EE checked in pax with tote/toolbox weighing 92lbs. EE marked the bag with two heavy tags. EE asked for assistance from another agent, to lift the heavy bag on to the ticket counter bag belt. EE stood with his left side closest to the belt. Together with the other agent, he lifted the bag with two hands moving the bag across his body from right to left. As EE moved the bag across his body, he heard a “pop” and immediately felt pain.","[1210pm, ee, checked, pax, totetoolbox, weighing, 92lbs, ee, marked, bag, two, heavy, tags, ee, asked, assistance, another, agent, lift, heavy, bag, ticket, counter, bag, belt, ee, stood, left, side, closest, belt, together, agent, lifted, bag, two, hands, moving, bag, across, body, right, left, ee, moved, bag, across, body, heard, pop, immediately, felt, pain]"
157,"On 14JAN22, Katy parked the Porsche at gate 37 to greet a passenger off DL3851 PDX-LAX. She parked at the left tail side of the aircraft with the rear of the Porsche facing south. After parking, ERMC arrived and backed up to ship 252 to service the lavatories. At approximately 1027p Katy and the passenger returned to the Porsche. The passenger got into the car and Katy proceeded to load the luggage into the trunk. As she was closing the trunk, the ABM lavatory service driver begins to pull away. Katy gets pushed up onto the side of the Porsche by the still connected lavatory service hose. She quickly pulls it away enough to get under it. Then proceeds to waive down the driver to stop.","[14jan22, katy, parked, porsche, gate, greet, passenger, dl3851, pdx, lax, parked, left, tail, side, aircraft, rear, porsche, facing, south, parking, ermc, arrived, backed, ship, service, lavatories, approximately, 1027p, katy, passenger, returned, porsche, passenger, got, car, katy, proceeded, load, luggage, trunk, closing, trunk, abm, lavatory, service, driver, begins, pull, away, katy, gets, pushed, onto, side, porsche, still, connected, lavatory, service, hose, quickly, pulls, away, enough, get, proceeds, waive, driver, stop]"
662,advised Josephine Calautti informed me this morning her left foot was slightly swollen and the bottom of her heel is in pain. The direct cause of this injury is unknown to employee as it was not contributed to any type of twisting or of that nature. On 06/18 @ approx. 11a Josephine was walking down the jet bridge on gate 21 when she felt sharp pains on the bottom of her heel through to her Achilles. This morning her lower left ankle became slightly swollen and she was hurting while putting pressure on heel while walking.,"[advised, josephine, calautti, informed, morning, left, foot, slightly, swollen, bottom, heel, pain, direct, cause, injury, unknown, employee, contributed, type, twisting, nature, approx, 11a, josephine, walking, jet, bridge, gate, felt, sharp, pains, bottom, heel, achilles, morning, lower, left, ankle, became, slightly, swollen, hurting, putting, pressure, heel, walking]"
495,Agent was assisting in lifting aisle chair with heavy passenger over the aircraft lip of door onto the jet bridge. She felt pain down her mid to lower back.,"[agent, assisting, lifting, aisle, chair, heavy, passenger, aircraft, lip, door, onto, jet, bridge, felt, pain, mid, lower, back]"
4,"Agent was assisting a passenger deplaning from remote ops spot 5C, he climbed up the steps and stopped at the second step to retrieve carry on item from passenger and he walked down the steps backwards. When he stepped down he felt his left ankle slightly roll but felt no pain. He continued working and clocked out at 2230. He woke up this morning (23NOV) and felt a sharp pain starting from his ankle bone shooting up his leg and his ankle was swollen as well. He is also limping as he walks","[agent, assisting, passenger, deplaning, remote, ops, spot, 5c, climbed, steps, stopped, second, step, retrieve, carry, item, passenger, walked, steps, backwards, stepped, felt, left, ankle, slightly, roll, felt, 

In [ ]:
#print(gensim.utils.simple_preprocess("AT 12:10pm EE checked in pax with tote/toolbox weighing 92lbs. EE marked the bag with two heavy tags. EE asked for assistance from another agent, to lift the heavy bag on to the ticket counter bag belt. EE stood with his left side closest to the belt. Together with the other agent, he lifted the bag with two hands moving the bag across his body from right to left. As EE moved the bag across his body, he heard a pop and immediately felt pain."))

In [ ]:
df['text'] = df['text'].str.replace(r'\W', ' ', case = False)
df['text'] = df['text'].str.replace(r'[.,?<>-]', '')

In [14]:
model = Word2Vec(sentences=tokenized_docs, vector_size=300, workers=1)
#wv = api.load('word2vec-google-news-300')
model.build_vocab(tokenized_docs, progress_per=1000)
#model = KeyedVectors.load_word2vec_format=('word2vec-google-news-300')

In [15]:
def vectorize(list_of_docs, model):
    """Generate vectors for list of documents using a Word Embedding

    Args:
        list_of_docs: List of documents
        model: Gensim's Word Embedding

    Returns:
        List of document vectors
    """
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                try:
                    vectors.append(model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features    


In [16]:
vectorized_docs = vectorize(tokenized_docs, model=model)
len(vectorized_docs), len(vectorized_docs[0])

(720, 300)

In [17]:
df["vectorized_docs"] =vectorized_docs

In [18]:
#model.similarity(w1="slip",w2="fall")

In [19]:
df.head()

,Description of Event,tokens,vectorized_docs
301,"AT 12:10pm EE checked in pax with tote/toolbox weighing 92lbs. EE marked the bag with two heavy tags. EE asked for assistance from another agent, to lift the heavy bag on to the ticket counter bag belt. EE stood with his left side closest to the belt. Together with the other agent, he lifted the bag with two hands moving the bag across his body from right to left. As EE moved the bag across his body, he heard a “pop” and immediately felt pain.","[1210pm, ee, checked, pax, totetoolbox, weighing, 92lbs, ee, marked, bag, two, heavy, tags, ee, asked, assistance, another, agent, lift, heavy, bag, ticket, counter, bag, belt, ee, stood, left, side, closest, belt, together, agent, lifted, bag, two, hands, moving, bag, across, body, right, left, ee, moved, bag, across, body, heard, pop, immediately, felt, pain]","[0.02078223, 0.28733596, 0.040639326, 0.14662439, 0.053288974, -0.23149274, 0.1421007, 0.5295764, 0.08089438, -0.105023816, 0.008849435, -0.32476515, 0.06760511, -0.123851515, -0.11228678, -0.18282238, 0.15936929, 0.045607127, 0.0934885, -0.014152064, -0.17246228, 0.0772793, 0.16892964, 0.06377067, 0.022525078, -0.0154312365, -0.34821892, 0.056510925, -0.16030723, -0.19473821, 0.06710984, -0.18251911, 0.07434252, -0.059736427, -0.07203949, 0.16165294, 0.055306293, -0.3178666, -0.12057288, 0.09298358, -0.101624966, 0.09543545, -0.055783104, -0.14444922, 0.20872512, 0.25495723, 0.06222806, 0.0071749776, -0.01998309, 0.15665336, 0.067331165, -0.013482443, -0.12223459, 0.06347043, -0.027874587, 0.23429559, 0.004018198, -0.112019174, 0.047091495, -0.006398526, -0.034146193, -0.08964167, 0.002964169, 0.047649346, -0.025708806, 0.09972803, -0.056284558, -0.0042402847, -0.11530547, -0.15914607, -0.037736095, 0.08804078, 0.30009148, -0.18848133, -0.043457553, 0.16498247, -0.30944487, 0.099162765, -0.08905923, 0.18599838, -0.2031958, -0.20715837, 0.08662451, 0.42640746, 0.055079065, -0.0001251785, -0.31122673, 0.056625664, 0.2427255, 0.14296466, 0.31089833, -0.14139344, 0.15720321, 0.000899923, 0.114472345, 0.2605898, 0.24237971, -0.011944372, -0.06241723, 0.15385789, ...]"
157,"On 14JAN22, Katy parked the Porsche at gate 37 to greet a passenger off DL3851 PDX-LAX. She parked at the left tail side of the aircraft with the rear of the Porsche facing south. After parking, ERMC arrived and backed up to ship 252 to service the lavatories. At approximately 1027p Katy and the passenger returned to the Porsche. The passenger got into the car and Katy proceeded to load the luggage into the trunk. As she was closing the trunk, the ABM lavatory service driver begins to pull away. Katy gets pushed up onto the side of the Porsche by the still connected lavatory service hose. She quickly pulls it away enough to get under it. Then proceeds to waive down the driver to stop.","[14jan22, katy, parked, porsche, gate, greet, passenger, dl3851, pdx, lax, parked, left, tail, side, aircraft, rear, porsche, facing, south, parking, ermc, arrived, backed, ship, service, lavatories, approximately, 1027p, katy, passenger, returned, porsche, passenger, got, car, katy, proceeded, load, luggage, trunk, closing, trunk, abm, lavatory, service, driver, begins, pull, away, katy, gets, pushed, onto, side, porsche, still, connected, lavatory, service, hose, quickly, pulls, away, enough, get, proceeds, waive, driver, stop]","[0.018185027, 0.24103673, 0.033950515, 0.12280439, 0.045673896, -0.19350724, 0.11986755, 0.44350573, 0.0674236, -0.08736359, 0.005948549, -0.27228317, 0.05648091, -0.10418014, -0.09363291, -0.15337855, 0.13365251, 0.038733337, 0.07763914, -0.01128005, -0.14527127, 0.0646305, 0.14240187, 0.05349175, 0.018938713, -0.012338214, -0.29150653, 0.046295498, -0.13494223, -0.16322286, 0.056475136, -0.15268292, 0.062198758, -0.05005803, -0.05995316, 0.13622993, 0.04683847, -0.26655528, -0.10114395, 0.078702785, -0.085272424, 0.07932398, -0.046804763, -0.12131594, 0.17498948, 0.21467352, 0.052842136, 0.0060765697,

In [ ]:
#similar_words = model.corpus_total_words('ergonomics')

In [23]:
word_vectors = model.wv
centroid = np.average([word_vectors[w] for w in ['agent', 'assisting', 'passenger']],axis=0)
centroid                       

array([ 0.02135359,  0.27528015,  0.03755542,  0.1407682 ,  0.05171154,
       -0.22131716,  0.13771528,  0.5068462 ,  0.07737219, -0.09927314,
        0.00730698, -0.31211594,  0.0642892 , -0.12010213, -0.1076315 ,
       -0.17503841,  0.1524787 ,  0.04305016,  0.08934892, -0.01259414,
       -0.16573702,  0.07508815,  0.16334355,  0.0623648 ,  0.02150093,
       -0.01418434, -0.3343953 ,  0.05211757, -0.15306143, -0.187674  ,
        0.06369116, -0.17471845,  0.06993175, -0.05852129, -0.0680497 ,
        0.15505655,  0.05287779, -0.30352917, -0.11293057,  0.08768768,
       -0.09864753,  0.08945364, -0.05320227, -0.13870548,  0.19737919,
        0.24444766,  0.05795242,  0.00734502, -0.01826408,  0.14981748,
        0.06492586, -0.01516534, -0.11391661,  0.05802617, -0.0274608 ,
        0.22703218,  0.00598049, -0.10588481,  0.04385154, -0.0051788 ,
       -0.03304135, -0.08638885,  0.00303132,  0.04693333, -0.02352046,
        0.09493224, -0.05498859, -0.00389691, -0.11005169, -0.15

In [28]:
# for label in label_names:
#     label_vectors = model[label]
# label_vectors

In [25]:
#print(model.wv['remote'])

In [29]:
label_names = ['ergonomics','sliptripfall','other']
vectorized_lbls = []
for label in label_names:
    if label in model.wv:
        #vectorized_lbls.append(model.wv[label])
        print(label)
        print(vectorized_lbls)

In [30]:
def embed(tokens, nlp):
    """Return the centroid of the embeddings for the given tokens.

    Out-of-vocabulary tokens are cast aside. Stop words are also
    discarded. An array of 0s is returned if none of the tokens
    are valid.

    """

    lexemes = (nlp.vocab[token] for token in tokens)

    vectors = np.asarray([
        lexeme.vector
        for lexeme in lexemes
        if lexeme.has_vector
        and not lexeme.is_stop
        and len(lexeme.text) > 1
    ])

    if len(vectors) > 0:
        centroid = vectors.mean(axis=0)
    else:
        width = nlp.meta['vectors']['width']  # typically 300
        centroid = np.zeros(width)

    return centroid

In [36]:
label_names = [['ergonomics'], ['sliptripfall'],['other']]
label_vectors = np.asarray([
    embed(label, nlp)
    for label in label_names
])
label_vectors.shape

(3, 300)

In [38]:
for label in label_names:
    print(label)

['ergonomics']
['sliptripfall']
['other']


In [37]:
label_vectors

array([[-3.23149991,  1.39170003,  0.61522001,  1.24890006,  1.30079997,
         0.34608001, -0.42649999,  2.47309995, -1.6142    ,  0.17183   ,
         3.1861999 ,  1.09389997, -5.05770016,  0.65094   , -0.44475999,
         4.7196002 ,  4.15439987,  0.76629001, -3.30240011, -1.20410001,
        -1.37010002,  0.41666999,  0.16638   ,  1.2306    , -0.95186001,
        -1.2507    , -1.23740005, -0.89238   , -1.50530005,  0.26585999,
        -1.1013    , -1.23699999,  2.91829991, -1.77740002, -0.085541  ,
         0.072318  ,  2.81509995, -0.15484001,  0.72745001,  1.45640004,
         2.03069997,  0.96011001, -1.02540004, -1.34809995,  0.19598   ,
        -0.19997001, -1.18040001, -2.09579992, -0.44082001, -5.31619978,
         1.90470004, -0.12857001, -0.35438001, -1.43079996,  0.14883   ,
         1.15690005, -1.73619998,  3.42499995, -1.4993    ,  0.93362999,
         0.69845998,  0.98834997, -0.26402   , -1.41960001,  1.2902    ,
         1.31850004, -0.86610001, -4.1704998 ,  0.9

In [40]:
#print([item for current in label_names for item in current.split(',')])

In [41]:
label_names

[['ergonomics'], ['sliptripfall'], ['other']]

In [42]:
label_names = ['ergonomics','sliptripfall','other']
label_vectors = np.asarray([embed(label.split(' '), nlp) for label in label_names.str])
label_vectors.shape

AttributeError: 'list' object has no attribute 'str'

In [ ]:
label_vectors

In [ ]:
label_vectors

In [ ]:
from sklearn.neighbors import NearestNeighbors
#neighbors=NearestNeighbors()

In [ ]:
neigh = NearestNeighbors(n_neighbors=1)
neigh.fit(label_vectors)


In [ ]:
closest_label = neigh.kneighbors(vectorized_docs, return_distance=False)[5, 0]

In [ ]:
label_names[closest_label]

In [ ]:
closest_label

In [ ]:
vectorized_lbs = vectorize(label_names, model=model)
len(vectorized_lbs), len(vectorized_lbs[0])

In [ ]:
vectorized_lbs[0]